In [1]:
import datetime, os, time, urllib2
from dateutil import tz

http://aqsdr1.epa.gov/aqsweb/aqstmp/airdata/download_files.html

https://ofmext.epa.gov/AQDMRS/ws/list?name=param&pc=CORE_HAPS&resource=rawData :

`12103	Arsenic (TSP) STP
12105	Beryllium (TSP) STP
12110	Cadmium (TSP) STP
12112	Chromium (TSP) STP
12115	Chromium VI (TSP) STP
12128	Lead (TSP) STP
12132	Manganese (TSP) STP
12136	Nickel (TSP) STP
14115	Chromium VI (TSP) LC
43218	1,3-Butadiene
43502	Formaldehyde
43503	Acetaldehyde
43505	Acrolein - Unverified
43509	Acrolein - Verified
43802	Dichloromethane
43803	Chloroform
43804	Carbon tetrachloride
43815	Ethylene dichloride
43817	Tetrachloroethylene
43818	1,1,2,2-Tetrachloroethane
43824	Trichloroethylene
43829	1,2-Dichloropropane
43830	trans-1,3-Dichloropropene
43831	cis-1,3-Dichloropropene
43843	Ethylene dibromide
43860	Vinyl chloride
45201	Benzene
82103	Arsenic PM10 STP
82105	Beryllium PM10 STP
82110	Cadmium PM10 STP
82112	Chromium PM10 STP
82128	Lead PM10 STP
82132	Manganese PM10 STP
82136	Nickel PM10 STP
82142	Mercury PM10 STP
88103	Arsenic PM2.5 LC
88105	Beryllium PM2.5 LC
88110	Cadmium PM2.5 LC
88112	Chromium PM2.5 LC
88128	Lead PM2.5 LC
88132	Manganese PM2.5 LC
88136	Nickel PM2.5 LC`

In [2]:
categories = {'44201': 'Ozone',
              '42401': 'SO2',
              '42101': 'CO',
              '42602': 'NO2',
              '88101': 'PM2.5', # federal reference method
              '88502': 'PM2.5', # non-FRM
              '81102': 'PM10',
              'SPEC':  None,
              'WIND':  None,
              'TEMP':  None,
              'PRESS': None,
              'RH_DP': None,
              'HAPS':  None,
              'VOCS':  None,
              'LEAD':  'Lead'}

years = range(1990, datetime.datetime.utcnow().year + 1)

src_dir = 'http://aqsdr1.epa.gov/aqsweb/aqstmp/airdata/'
dest_dir = 'EPA-AirData/'

# Last-Modified: Thu, 19 Jun 2014 17:28:44 GMT
# If-Modified-Since: Thu, 19 Jun 2014 17:28:44 GMT
def mirror(src, dest):
    headers = {}
    try:
        dest_timestamp = datetime.datetime.fromtimestamp(os.path.getmtime(dest), tz.tzutc())
        headers['If-Modified-Since'] = dest_timestamp.strftime('%a, %d %b %Y %H:%M:%S GMT')
    except:
        pass
    
    try:
        response = urllib2.urlopen(urllib2.Request(src, headers=headers))
        last_modified_str = response.info().getheader('Last-Modified')
        last_modified = datetime.datetime.strptime(last_modified_str, "%a, %d %b %Y %H:%M:%S GMT")
        last_modified_epoch = (last_modified - datetime.datetime(1970, 1, 1)).total_seconds()
        data = response.read()
    except urllib2.HTTPError, e:
        if e.code == 304:
            print 'Skipping %s since unmodified' % dest
            return
        print 'Exception while fetching %s' % src
        raise
    open(dest + '.tmp', 'w').write(data)
    os.utime(dest + '.tmp', (last_modified_epoch, last_modified_epoch))
    os.rename(dest + '.tmp', dest)
        
    printable_modtime = last_modified.strftime('%Y-%m-%d %H:%M:%S')
    print 'STATUS(SUCCESS) Mirrored %s (last modtime %s, %d bytes) to %s' % (src, printable_modtime, len(data), dest)

In [ ]:
try:
    os.mkdir(dest_dir)
except:
    pass

for year in years:
    for category in categories:
        filename = 'hourly_%s_%d.zip' % (category, year)
        mirror(src_dir + filename, dest_dir + filename)